In [5]:
%matplotlib widget
import imageio
import numpy as np
from pathlib import Path
from omegaconf import OmegaConf
import os
from IPython.display import display, Image
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn import linear_model
import torch.nn.functional as F
import torch
import yaml

import sys
base_path = Path(os.path.abspath('')).parent
print(base_path)
sys.path.append(str(base_path))
import torch
from dataLoader import dataset_dict

def disp_im(im):
    plt.close()
    plt.cla()
    fig = plt.imshow(im)
    plt.close()
    display(fig.figure)

/home/dronelab/nmf


In [6]:

def get_stats(p):
    stats_files = ['stats_augnle.yaml', 'stats_augnl.yaml', 'stats_augn.yaml', 'stats_aug.yaml', 'stats.yaml']
    stat_file = [s for s in stats_files if (p / "imgs_test_all" / s).exists()]
    stat_file = stat_file[0] if len(stat_file) else None
    if stat_file is None:
        return None
    data_path = p / "imgs_test_all" / stat_file
    with data_path.open('r') as f:
        data = yaml.safe_load(f)
    return data

In [7]:

# log_dir = Path("../log") / "tensorf"
# log_dir = Path("../log") / "fixedmip128"
# log_dir = Path("../log") / "singlebounce_samen"
# exps = ["noprednorms_nl0_conserve_pb0", 'fixedmip128', 'tensorf', 'singlebounce_samen']
# exps = ['interpdiffuse', 'interpdiffuse_flipnorm', 'fixedmip128', 'tensorf', 'singlebounce_samen']
# exps = ['fixedmip']
# exps = ['nodiffuse', 'fixedmip', 'noconv']
# exps = ['test_exps']
exps = ['fresnel_nointim', 'fresnel_nobounce', 'fresnel', 'lambda', 'fresnel_nonumer']
datadir = "/optane/nerf_datasets"
# datadir = "/data"
for exp in exps:
    log_dir = Path("../log") / exp
    for p in log_dir.glob("*"):
        print(p)
        pano_path = p / "imgs_test_all" / "envmaps" / "pano.exr"
        mapped_pano_path = p / "imgs_test_all" / "envmaps" / "mapped_pano.png"
        old_data = get_stats(p)
        new_data_path = p / "imgs_test_all" / "stats_augnle2.yaml"
        config = OmegaConf.load(p / 'config.yaml')
        dname = Path(config['dataset']['scenedir']).name

        white_bg = config.dataset.white_bg if hasattr(config.dataset, 'white_bg') else True
        dataset = dataset_dict[config.dataset.dataset_name]
        test_dataset = dataset(os.path.join(datadir, config.dataset.scenedir), split='test',
                            downsample=config.dataset.downsample_train, is_stack=True, white_bg=white_bg, is_testing=True)
        norm_errs = []
        for idx in range(len(test_dataset)):
            pnorms = torch.as_tensor(imageio.imread(p / "imgs_test_all" / "world_normal" / f"{idx:03d}.png")).float()
            pnorms = (pnorms - 128) / 127
            # pnorms = pnorms / torch.linalg.norm(pnorms, dim=-1, keepdim=True)
            try:
                gt_normal = test_dataset.get_normal(idx)
            except:
                ic("HI")
                break

            norm_err = torch.arccos((pnorms * gt_normal).sum(dim=-1).clip(min=1e-8, max=1-1e-8)) * 180/np.pi
            norm_err[torch.isnan(norm_err)] = 0

            # f, axs = plt.subplots(2, 2)
            # axs[0, 0].imshow(gt_normal)
            # axs[1, 0].imshow(norm_err)
            # axs[0, 1].imshow((pnorms * gt_normal).sum(dim=-1).clip(min=1e-8, max=1-1e-8))
            # axs[1, 1].imshow(norm_err)
            # plt.show()
            norm_err *= test_dataset.acc_maps[idx].squeeze(-1)
            # print(norm_err.mean())
            # break

            norm_errs.append(norm_err.mean())
        # break
        with new_data_path.open('w') as f:
            old_data['norm_err'] = float(sum(norm_errs) / len(norm_errs)) if len(norm_errs) > 0 else 0
            yaml.dump(old_data, f)

../log/fresnel_nointim/toaster_v1_fresnel_nointim
Original Image size: 800 x 800
Image size: 800 x 800


ic| self.fx: 1250.0000504168488, self.fy: 1250.0000504168488
/home/dronelab/miniconda3/envs/31/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
ic| self.intrinsics: tensor([[1.2500e+03, 0.0000e+00, 4.0000e+02],
                             [0.0000e+00, 1.2500e+03, 4.0000e+02],
                             [0.0000e+00, 0.0000e+00, 1.0000e+00]])
Loading data test (200): 100%|██████████| 200/200 [00:04<00:00, 40.64it/s]
/tmp/ipykernel_29054/1657540178.py:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pnorms = torch.as_tensor(imageio.imre

../log/fresnel_nointim/chair_v1_fresnel_nointim
Original Image size: 800 x 800
Image size: 800 x 800


Loading data test (200): 100%|██████████| 200/200 [00:04<00:00, 43.85it/s]
ic| self.fx: 1111.1110311937682, self.fy: 1111.1110311937682
ic| self.intrinsics: tensor([[1.1111e+03, 0.0000e+00, 4.0000e+02],
                             [0.0000e+00, 1.1111e+03, 4.0000e+02],
                             [0.0000e+00, 0.0000e+00, 1.0000e+00]])


../log/fresnel_nointim/ficus_v1_fresnel_nointim
Original Image size: 800 x 800
Image size: 800 x 800


...

Loading data test (200): 100%|██████████| 200/200 [00:04<00:00, 45.91it/s]
ic| self.fx: 1111.1110311937682, self.fy: 1111.1110311937682
ic| self.intrinsics: tensor([[1.1111e+03, 0.0000e+00, 4.0000e+02],
                             [0.0000e+00, 1.1111e+03, 4.0000e+02],
                             [0.0000e+00, 0.0000e+00, 1.0000e+00]])


../log/fresnel/mic_v1_fresnel
Original Image size: 800 x 800
Image size: 800 x 800


Loading data test (200): 100%|██████████| 200/200 [00:03<00:00, 52.60it/s]
ic| self.fx: 1250.0000504168488, self.fy: 1250.0000504168488
ic| self.intrinsics: tensor([[1.2500e+03, 0.0000e+00, 4.0000e+02],
                             [0.0000e+00, 1.2500e+03, 4.0000e+02],
                             [0.0000e+00, 0.0000e+00, 1.0000e+00]])


../log/fresnel/car_v1_fresnel
Original Image size: 800 x 800
Image size: 800 x 800


Loading data test (200):  54%|█████▍    | 108/200 [00:02<00:01, 50.67it/s]